In [2]:
!nvidia-smi

Sat Jul 13 05:37:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090         Off| 00000000:01:00.0 Off |                  Off |
| 45%   63C    P2              301W / 450W|  24091MiB / 24564MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
import torch
device_str = 'cuda:3'
print('device used: ', device_str)
device = torch.device(device_str)

device used:  cuda:3


In [4]:
def print_memory_status():
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    reserved_memory = torch.cuda.memory_reserved(device)
    free_memory = total_memory - allocated_memory
    
    print(f"Total memory: {total_memory / (1024 ** 2):.2f} MB")
    print(f"Allocated memory: {allocated_memory / (1024 ** 2):.2f} MB")
    print(f"Reserved memory: {reserved_memory / (1024 ** 2):.2f} MB")
    print(f"Free memory: {free_memory / (1024 ** 2):.2f} MB")
    print("-" * 40)

In [14]:
print_memory_status()

Total memory: 24217.31 MB
Allocated memory: 0.00 MB
Reserved memory: 1910.00 MB
Free memory: 24217.31 MB
----------------------------------------


In [15]:
torch.cuda.synchronize()  # 确保 CUDA 操作完成


RuntimeError: CUDA error: out of memory

In [10]:
print("Initial memory status:")
print_memory_status()

tensor_size = (10000, 10000)
print("Allocating large tensors to CUDA device...")
tensors = [torch.randn(tensor_size, device=device) for _ in range(5)]
print("Memory status after allocation:")
print_memory_status()

print("Releasing tensors from CUDA device...")
del tensors
torch.cuda.empty_cache()

print("Memory status after releasing tensors:")
print_memory_status()

Initial memory status:
Total memory: 24217.31 MB
Allocated memory: 0.00 MB
Reserved memory: 0.00 MB
Free memory: 24217.31 MB
----------------------------------------
Allocating large tensors to CUDA device...
Memory status after allocation:
Total memory: 24217.31 MB
Allocated memory: 1910.00 MB
Reserved memory: 1910.00 MB
Free memory: 22307.31 MB
----------------------------------------
Releasing tensors from CUDA device...


RuntimeError: CUDA error: out of memory

In [7]:
another_tensor = torch.randn(tensor_size, device=device)
print("Memory status after another allocation:")
print_memory_status()

Memory status after another allocation:
Total memory: 24209.12 MB
Allocated memory: 382.00 MB
Reserved memory: 1910.00 MB
Free memory: 23827.12 MB
----------------------------------------


In [ ]:
config = argparse.Namespace(act='ELU', action_repeat=2, actor_disc=5, actor_dist='trunc_normal', actor_entropy='1e-4', actor_grad_clip=100, actor_init_std=1.0, actor_layers=4, actor_lr=8e-05, actor_min_std=0.1, actor_outscale=0.0, actor_state_entropy=0.0, actor_temp=0.1, batch_length=50, batch_size=50, behavior_stop_grad=True, clip_rewards='identity', cnn_depth=32, collect_dyn_sample=True, dataset_size=0, debug=False, decoder_kernels=(5, 5, 6, 6), decoder_thin=True, device='cuda:0', disag_action_cond=False, disag_layers=4, disag_log=True, disag_models=10, disag_offset=1, disag_target='stoch', disag_units=400, discount=0.99, discount_lambda=0.95, discount_layers=3, discount_scale=1.0, dyn_cell='gru', dyn_deter=200, dyn_discrete=0, dyn_hidden=200, dyn_input_layers=1, dyn_mean_act='none', dyn_min_std=0.1, dyn_output_layers=1, dyn_rec_depth=1, dyn_shared=False, dyn_std_act='sigmoid2', dyn_stoch=50, dyn_temp_post=True, encoder_kernels=(4, 4, 4, 4), envs=1, eval_every=10000.0, eval_noise=0.0, eval_state_mean=False, evaldir=None, expl_amount=0.0, expl_behavior='greedy', expl_extr_scale=0.0, expl_gifs=False, expl_intr_scale=1.0, expl_until=0, future_entropy=False, grad_clip=100, grad_heads=('image', 'reward'), grayscale=False, imag_gradient='dynamics', imag_gradient_mix='0.1', imag_horizon=15, imag_sample=True, kl_balance='0.8', kl_forward=False, kl_free='1.0', kl_scale='1.0', log_every=10000.0, logdir=None, model_lr=0.0003, offline_evaldir='', offline_traindir='', opt='adam', opt_eps=1e-05, oversample_ends=False, precision=16, pred_discount=False, prefill=2500, pretrain=100, reset_every=0, reward_layers=2, reward_scale=1.0, seed=0, size=(64, 64), slow_actor_target=True, slow_target_fraction=1, slow_target_update=100, slow_value_target=True, steps=10000000.0, task='dmc_walker_walk', time_limit=1000, train_every=5, train_steps=1, traindir=None, units=400, value_decay=0.0, value_grad_clip=100, value_head='normal', value_layers=3, value_lr=8e-05, weight_decay=0.0)

config.logdir = '摄影师'
config.eval_every = 5e3
config.log_every = 5e3
config.steps = 1e5 
state_dict_path = 'log/latest_model.pt'
device = 'cuda:2'

In [ ]:
class Dreamer(nn.Module):

  def __init__(self, config, logger, dataset):
    super(Dreamer, self).__init__()
    self._config = config
    self._logger = logger
    self._should_log = tools.Every(config.log_every)
    self._should_train = tools.Every(config.train_every)
    self._should_pretrain = tools.Once()
    self._should_reset = tools.Every(config.reset_every)
    self._should_expl = tools.Until(int(
        config.expl_until / config.action_repeat))
    self._metrics = {}
    self._step = count_steps(config.traindir)
    # Schedules.
    config.actor_entropy = (
        lambda x=config.actor_entropy: tools.schedule(x, self._step))
    config.actor_state_entropy = (
        lambda x=config.actor_state_entropy: tools.schedule(x, self._step))
    config.imag_gradient_mix = (
        lambda x=config.imag_gradient_mix: tools.schedule(x, self._step))
    self._dataset = dataset
    self._wm = models.WorldModel(self._step, config)
    self._task_behavior = models.ImagBehavior(
        config, self._wm, config.behavior_stop_grad)
    reward = lambda f, s, a: self._wm.heads['reward'](f).mean
    self._expl_behavior = dict(
        greedy=lambda: self._task_behavior,
        random=lambda: expl.Random(config),
        plan2explore=lambda: expl.Plan2Explore(config, self._wm, reward),
    )[config.expl_behavior]()

  def __call__(self, obs, reset, state=None, reward=None, training=True):
    step = self._step
    if self._should_reset(step):
      state = None
    if state is not None and reset.any():
      mask = 1 - reset
      for key in state[0].keys():
        for i in range(state[0][key].shape[0]):
          state[0][key][i] *= mask[i]
      for i in range(len(state[1])):
        state[1][i] *= mask[i]
    if training and self._should_train(step):
      steps = (
          self._config.pretrain if self._should_pretrain()
          else self._config.train_steps)
      for _ in range(steps):
        self._train(next(self._dataset))
      if self._should_log(step):
        for name, values in self._metrics.items():
          self._logger.scalar(name, float(np.mean(values)))
          self._metrics[name] = []
        openl = self._wm.video_pred(next(self._dataset))
        self._logger.video('train_openl', to_np(openl))
        self._logger.write(fps=True)

    policy_output, state = self._policy(obs, state, training)

    if training:
      self._step += len(reset)
      self._logger.step = self._config.action_repeat * self._step
    return policy_output, state

  def _policy(self, obs, state, training):
    if state is None:
      batch_size = len(obs['image'])
      latent = self._wm.dynamics.initial(len(obs['image']))
      action = torch.zeros((batch_size, self._config.num_actions)).to(self._config.device)
    else:
      latent, action = state
    embed = self._wm.encoder(self._wm.preprocess(obs))
    latent, _ = self._wm.dynamics.obs_step(
        latent, action, embed, self._config.collect_dyn_sample)
    if self._config.eval_state_mean:
      latent['stoch'] = latent['mean']
    feat = self._wm.dynamics.get_feat(latent)
    if not training:
      actor = self._task_behavior.actor(feat)
      action = actor.mode()
    elif self._should_expl(self._step):
      actor = self._expl_behavior.actor(feat)
      action = actor.sample()
    else:
      actor = self._task_behavior.actor(feat)
      action = actor.sample()
    logprob = actor.log_prob(action)
    latent = {k: v.detach()  for k, v in latent.items()}
    action = action.detach()
    if self._config.actor_dist == 'onehot_gumble':
      action = torch.one_hot(torch.argmax(action, dim=-1), self._config.num_actions)
    action = self._exploration(action, training)
    policy_output = {'action': action, 'logprob': logprob}
    state = (latent, action)
    return policy_output, state

  def _exploration(self, action, training):
    amount = self._config.expl_amount if training else self._config.eval_noise
    if amount == 0:
      return action
    if 'onehot' in self._config.actor_dist:
      probs = amount / self._config.num_actions + (1 - amount) * action
      return tools.OneHotDist(probs=probs).sample()
    else:
      return torch.clip(torchd.normal.Normal(action, amount).sample(), -1, 1)
    raise NotImplementedError(self._config.action_noise)

  def _train(self, data):
    metrics = {}
    post, context, mets = self._wm._train(data)
    metrics.update(mets)
    start = post
    if self._config.pred_discount:  # Last step could be terminal.
      start = {k: v[:, :-1] for k, v in post.items()}
      context = {k: v[:, :-1] for k, v in context.items()}
    reward = lambda f, s, a: self._wm.heads['reward'](
        self._wm.dynamics.get_feat(s)).mode()
    metrics.update(self._task_behavior._train(start, reward)[-1])
    if self._config.expl_behavior != 'greedy':
      if self._config.pred_discount:
        data = {k: v[:, :-1] for k, v in data.items()}
      mets = self._expl_behavior.train(start, context, data)[-1]
      metrics.update({'expl_' + key: value for key, value in mets.items()})
    for name, value in metrics.items():
      if not name in self._metrics.keys():
        self._metrics[name] = [value]
      else:
        self._metrics[name].append(value)